# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-03-31 00:00:00  to  2020-04-01 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,72,92,20,27.777778
1,02,144,164,20,13.888889
2,03,36,38,2,5.555556
3,04,20,22,2,10.000000
4,05,42,52,10,23.809524


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2434,2633,199,8.175842
92,92,1446,1638,192,13.278008
94,94,1170,1368,198,16.923077
93,93,964,1173,209,21.680498
69,69,1054,1114,60,5.692600


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -18  Mean:  19.510416666666668  Max:  209


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -20.0  Mean %:  10.975197127957912  Max %:  60.0


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
6,07,87,85,-2,-2.298851
28,2A,89,88,-1,-1.123596
29,2B,10,8,-2,-20.000000
31,31,219,216,-3,-1.369863
33,33,300,296,-4,-1.333333
42,42,415,414,-1,-0.240964
66,66,122,111,-11,-9.016393
70,70,69,68,-1,-1.449275
71,71,191,173,-18,-9.424084
72,72,101,99,-2,-1.980198


#### <br> Departments with highest evolution of hospitalizations (total number)

In [11]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
93,93,964,1173,209,21.680498
75,75,2434,2633,199,8.175842
94,94,1170,1368,198,16.923077
92,92,1446,1638,192,13.278008
78,78,706,815,109,15.439093


#### <br> Departments with highest rate of hospitalizations (%)

In [12]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
46,46,5,8,3,60.000000
65,65,28,39,11,39.285714
82,82,8,11,3,37.500000
30,30,55,74,19,34.545455
8,09,12,16,4,33.333333


#### <br> Focus on some departments

In [13]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2434,2633,199,8.175842


In [14]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,191,173,-18,-9.424084


In [15]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,219,216,-3,-1.369863
